<center>

*******************************************************************************************
<font size="4">
BATCH PROCESS &nbsp; &nbsp; <br>
RENAME FILES &nbsp; &nbsp; <br>
</font>

##### 1 July 2025

##### Juan Ignacio Mendoza Garay

*******************************************************************************************

</center>

#### INFORMATION:


* Description:

    Rename all files in a folder preserving the extensions.
    Strings can be replaced using a reference CSV file.
    Useful, for example, to anonymise data files.

* Instructions:

    Edit the values indicated with an arrow like this: <--- \
    Run the program, close your eyes and hope for the best.

*******************************************************************************************


In [1]:
files_dir = 'C:/Users/Hoppy/Documents/my_files' # <--- directory of files to rename

replace_str = ['_raw','_rn'] # <--- replace string in all file names [old,new] or nested lists

csv_fn = 'C:/Users/Hoppy/Documents/old_new_str.csv' # <--- path to CSV file, or None
csv_header = True         # <--- whether the CSV file has a header
csv_cols_replace = [0,1]  # <--- columns with strings to replace [old,new]; int, or str if csv_header = True

prefix = 'A'        # <--- prefix for all new file names
suffix = 'new'      # <--- suffix for all new file names
add_str = [0,[1,2]] # <--- add to [prefix,suffix] or nested lists (all unique values)
# 0 = consecutive number
# 1 = current date (YearMonthDay)
# 2 = random string

verbose = True # <--- display information such as old and new file names
test = True    # <--- run process except renaming files

In [ ]:
# Process replace_str:
if not isinstance(replace_str[0],list): replace_str = [replace_str]

# Process CSV file:
if csv_fn:
    import csv
    csv_dict = {}
    with open(csv_fn) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        if csv_header:
            csv_header = next(csv_reader)
            if verbose: print('CSV file header:\n',csv_header,'\n')
            for i_c,c in enumerate(csv_cols_replace):
                if isinstance(c,str):
                    csv_cols_replace[i_c] = csv_header.index(c)
        for i_row,row in enumerate(csv_reader):
            csv_dict[row[csv_cols_replace[0]]] = row[csv_cols_replace[1]]

In [ ]:
import os
files_in_folder = os.listdir(files_dir)

# Check added text:
assert len(add_str), '"add_str" should be a list with two elements'
pref_suff = ['','']
if prefix: pref_suff[0] = '_'+prefix
if suffix: pref_suff[1] = '_'+suffix
sep_pa = ['','']
sep_ch = ['','']
astr_vals = []
for i_astr, astr in enumerate(add_str):
    if pref_suff[i_astr] or (astr is not None): sep_pa[i_astr] = '_'
    if (astr is not None) and  isinstance(astr,list) and (len(astr)>1): sep_ch[i_astr] = '_'
    if not isinstance(astr,list): add_str[i_astr] = [astr]
    for v in add_str[i_astr]:
        if v is not None:
            assert v not in astr_vals, '"add_str" should not have repeated values'
        astr_vals.append(v)
        if v==0: pass
        elif v==1: from datetime import datetime
        elif v==2:
            import string, random
            ran_str = []
            allchar = string.ascii_uppercase + string.digits
            n_files=len(files_in_folder)
            n_el = int(n_files/5)
            if n_el<3: n_el = 3
            n_iter = n_files*10 # <--- increase if needed
            for i_iter in range(n_files*10):
                ran_str.append(''.join(random.choice(allchar) for _ in range(n_el)))
                if len(set(ran_str)) == n_files: break
            if len(ran_str) < n_files:
                rng_warning = ''.join(['The number of generated random strings is less than the ',
                                       'number of files. Try running the program again and ',
                                       'increasing the value of "n_iter"'])
                raise Exception(rng_warning)
        else: assert v in [None,''], 'invalid value for "add_str"'

In [ ]:
if test: print('Warning: This is a test; files have not been renamed.\n')

for i_fn, src_fn in enumerate(files_in_folder):
    full_src = files_dir+'\\'+src_fn
    src_fn_ne, this_ext  = os.path.splitext(src_fn)
    if replace_str:
        for rs in replace_str:
            src_fn_ne = src_fn_ne.replace(rs[0],rs[1])
    if csv_fn:
        try:
            old_substr = next(old_substr for old_substr in csv_dict if old_substr in src_fn)
        except:
            raise Exception('Substring not found. Check file names and CSV file.')
        src_fn_ne = src_fn_ne.replace(old_substr,csv_dict[old_substr])
    pref_suff_p = ['','']
    for i_ps, ps in enumerate(pref_suff):
        if add_str[i_ps] is not None:
            for i_v, v in enumerate(add_str[i_ps]):
                if i_v == 0: this_sep_ch = ''
                else: this_sep_ch = sep_ch[i_ps]
                if v == 0:
                    pref_suff_p[i_ps] =  pref_suff_p[i_ps] + f'{this_sep_ch}{i_fn}'
                elif v == 1:
                    d = datetime.today().strftime('%Y%m%d')
                    pref_suff_p[i_ps] =  pref_suff_p[i_ps] + f'{this_sep_ch}{d}'
                elif v == 2:
                    pref_suff_p[i_ps] = pref_suff_p[i_ps] + f'{this_sep_ch}{ran_str[i_fn]}'

        if i_ps==0: pref_suff_p[i_ps] = pref_suff_p[i_ps] + ps + sep_pa[0]
        else: pref_suff_p[i_ps] =  sep_pa[1] + pref_suff_p[i_ps] + ps

    output_fn = pref_suff_p[0] + src_fn_ne + pref_suff_p[1] + this_ext
    if verbose or test: print(f'OLD: {src_fn}\nNEW: {output_fn}\n')
    assert src_fn != output_fn, 'No file name change. Change parameters.'
    full_dst = files_dir+'\\'+output_fn
    if not test: os.rename(full_src, full_dst)
    break

print('done')